# INSTALL

``` 
sudo docker pull dgraph/dgraph
sudo curl -L "https://github.com/docker/compose/releases/download/1.23.2/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose
docker-compose --version
```

# RUN dGraph

Make sure the file, docker-compose.yml from https://docs.dgraph.io/get-started#step-2-run-dgraph, is present in the working directory.

```
sudo docker-compose up -d
```
The UI can be accessed at http://localhost:8000/

```
sudo docker-compose down
```
shuts down the servers

## Setup Virtual environment

```
python3 -m venv pydGraph-flask-venv
source pydGraph-flask-venv/bin/activate
pip install dgraph
pip install Flask
pip install pydgraph
pip install pymysql
pip install nltk
pip install spacy
python -m spacy download en_core_web_sm
python -m spacy download en_core_web_md
deactivate
```

nltk_data has been downloaded and is available in the home folder

## To export from local

updated the docker-compose.yml file adding 

```--whitelist 172.18.0.1:172.99.0.1``` 

to line  ```command: dgraph alpha --my=server:7080 --lru_mb=2048 --zero=zero:5080 ```


after starting all the three servers using the above docker-compose command, run

```sudo curl localhost:8080/admin/export```

Then find out the docker container ID of aplha server with

```docker ps```

use it in

```docker cp fa24fb240869:/dgraph/export .```

to copy the export folder to the export folder in the working directory

change the permission of the copied export(specify the export file name correctly)


```sudo chmod -R a+rwx export/dgraph.r2610001.u1112.0604/```

or

in case you dont want to update the docker-compose.yml with whitelist IP, then 

Go inside that container and execute the export command there.


```
sudo docker exec -it “container-id” bash
curl localhost:8080/admin/export
exit
```

and then copy the export files to host as before


## To import to server
(not tested)

```dgraph live -r <path-to-gzipped-RDf-or-JSON-file> -s <path-to-schema-file> -a <dgraph-alpha-address:grpc_port> -z <dgraph-zero-address:grpc_port>```


## Neo4j Cypher Queries Rewritten in GraphQL+-

To match a path from bible node to words of a verse

```
MATCH p=((bib:BIBLE)<-[:BELONGS_TO]-(b:BOOK)<-[:BELONGS_TO]-(c:CHAPTER)<-[:BELONGS_TO]-(v:VERSE)<-[:BELONGS_TO]-(w:WORD))
WHERE bib.name='Hindi_IRV4' and b.number='40' and c.number='5' and v.number='3' 
RETURN p
```

```
{ verse_path (func: uid(0x178ae)){
    bible
    ~belongsTo @filter(eq(bookNumber,40)){
        book
        bookName
        ~belongsTo @filter(eq(chapter,5)){
            chapter
            ~belongsTo @filter(eq(verse,3)){
                verse
                lid
                ~belongsTo {
                    position
                    word
                    }
                 }
             }
         }
     }
}
```

To retrieve just the text, from the above verse

```
MATCH p=((bib:BIBLE)<-[:BELONGS_TO]-(b:BOOK)<-[:BELONGS_TO]-(c:CHAPTER)<-[:BELONGS_TO]-(v:VERSE)<-[:BELONGS_TO]-(w:WORD))
WHERE bib.name='Hindi_IRV4' and b.number='40' and c.number='5' and v.number='3' 
RETURN w.literalform ORDER BY w.position
```

```
{ words (func: uid(0x178ae)) @normalize{
    bible
    ~belongsTo @filter(eq(bookNumber,40)){
        book
        bookName
        ~belongsTo @filter(eq(chapter,5)){
            chapter
            ~belongsTo @filter(eq(verse,3)){
                verse
                lid
                ~belongsTo {
                    position
                    word: word
                    }
                 }
             }
         }
     }
}
```

To view UGNT bible with links to Translation words and Strong Numbers

```
MATCH p=((bib:BIBLE)<-[:BELONGS_TO]-(:BOOK)<-[:BELONGS_TO]-(:CHAPTER)<-[:BELONGS_TO]-(:VERSE)<-[:BELONGS_TO]-(w:WORD)-[:IS_TW]->(:TRANSLATION_WORD)-[:BELONGS_TO]->(:DICTIONARY))
WITH p,w
MATCH (w)-[:HAS_LEMMA]->(n:STRONG_NUM)-[:BELONGS_TO]->(d:DICTIONARY)
RETURN p,n,d limit 5
```

```
{ verse_path (func: uid(0x12cac)) @cascade{
    bible
    ~belongsTo @filter(eq(bookNumber,40)){
        book
        bookNumber
        ~belongsTo @filter(eq(chapter,5)){
            chapter
            ~belongsTo @filter(eq(verse,3)){
                verse
                lid
                ~belongsTo {
                    position
                    word
                    lemma {
                        StrongsNumber
                        englishWord
                        }
                    tw {
                        translationWord
                        wordForms
                        }
                    
                    }
                 }
             }
         }
     }
}
```

To find the occurences of a word/strongs in a bible

```
MATCH p=((bib:BIBLE)<-[:BELONGS_TO]-(b:BOOK)<-[:BELONGS_TO]-(c:CHAPTER)<-[:BELONGS_TO]-(v:VERSE)<-[:BELONGS_TO]-(w:WORD)-[:HAS_LEMMA]->(s:STRONG_NUM))
WHERE s.id = 2424 and bib.name='UGNT4'
RETURN p LIMIT 5
```

```
{ strong_occur (func: eq(StrongsNumber,2424)){
    uid
    StrongsNumber
    englishWord
    ~lemma {
        word
        position
        belongsTo {
            verse
            lid
            belongsTo {
                chapter
                belongsTo {
                    book
                    bookNumber
                    belongsTo {
                        bible
                        }
                    }
                }
            }
       }
    }
}
 ```

To see how a verse is in various bibles

```
MATCH p=((bib:BIBLE)<-[:BELONGS_TO]-(b:BOOK)<-[:BELONGS_TO]-(c:CHAPTER)<-[:BELONGS_TO]-(v:VERSE)<-[:BELONGS_TO]-(w:WORD))
WHERE b.number='40' and c.number='5' and v.number='3' 
RETURN p
```

```
{ verse_path (func: has(bible)){
    bible
    ~belongsTo @filter(eq(bookNumber,40)){
        book
        bookName
        ~belongsTo @filter(eq(chapter,5)){
            chapter
            ~belongsTo @filter(eq(verse,3)){
                verse
                lid
                ~belongsTo {
                    position
                    word
                    }
                 }
             }
         }
     }
}
```

To find all the translations of a particular word/lemma

```
MATCH (w:WORD)-[:HAS_LEMMA]->(s:STRONG_NUM),
(a:WORD)-[:ALIGNS_TO]-(w)
WHERE s.id = 2424
RETURN w, a limit 3
```

```
{ abraham_translations( func: eq(StrongsNumber,11)) {
    uid
    englishWord
    StrongsNumber
    ~lemma {
        uid
        word
        ~alignsTo {
            uid
            word
            }
        }
   }
}

```


To find the names from the Bibles without names marked

```
match (w:WORD)-[:ALIGNS_TO]-(:WORD)-[:IS_TW]-(tw:TRANSLATION_WORD) where tw.type='names' with distinct tw , w return w limit 10
```

```
{ names( func: eq(twType,"names")) @normalize{
    uid
    eng: englishWord
    StrongsNumber
    ~tw {
        uid
        word
        ~alignsTo {
            word: word
            }
        }
   }
}
```